# Algorithm implementation
We have a clean dataset, we can now find frequent itemset in the data.

### --> efficient apriori version (slower)

## Libraries

In [10]:
import pandas as pd
import math
import plotly.express as px     # used to plot the data
import ast                      # used to transform string into list
from collections import Counter # used to count the occurrences of hashtags
from statistics import mean
import string                   # used to remove punctuation from text in an efficient way
from tqdm import tqdm
import datetime
import time
import json

# EFFICIENT APRIORI
from efficient_apriori import apriori


print(" > All libreries imported")

 > All libreries imported


## Import the clean dataset

In [11]:
df_path = "../data/clean_df/clean_df.pkl"
df = pd.read_pickle(df_path)

## APriori on all days

### 1. Compute APriori algorithm for each day of the dataset

In [12]:
start = time.time()

curr_day = datetime.date(2020, 7, 24)  # first day of the data
last_day = datetime.date(2020, 8, 30)  # last day of the data
results_list = []

# iterate on every day of the dataset (until we reach the last one)
while (curr_day <= last_day):

    # prepare the list of item of the day
    texts_for_apriori = []
    for t in df.loc[df['date'] == curr_day]["text"]:
        texts_for_apriori.append(t)

    print(" >> Computing APriori on day:", curr_day, "| Total tweets:", len(texts_for_apriori))


    # computation of APriori [only if Total tweets > 0]
    if len(texts_for_apriori) > 0:
        itemsets, rules = apriori(texts_for_apriori, min_support=0.012,  min_confidence=0.8)

        # Save the results in a more convenient way
        list_of_groups = []
        for item in itemsets.values():
            list_of_groups.append(item)   # there are always groups of 1, 2 and 3 words. Sometimes more.

        threshold = 0.015
        daily_groups = []
        for group in list_of_groups:
            daily_groups.append({k: v/len(texts_for_apriori) for k, v in group.items() if v/len(texts_for_apriori) > threshold})
        
        results_list.append([curr_day, daily_groups])

    else:
        # Save the day and an empty list
        results_list.append([curr_day, [{}]])

    # add 1 day and repeat the procedure
    curr_day = curr_day + datetime.timedelta(days=1)

end = time.time()
eff_apriori_time = end - start

 >> Computing APriori on day: 2020-07-24 | Total tweets: 295
 >> Computing APriori on day: 2020-07-25 | Total tweets: 16881
 >> Computing APriori on day: 2020-07-26 | Total tweets: 7500
 >> Computing APriori on day: 2020-07-27 | Total tweets: 7500
 >> Computing APriori on day: 2020-07-28 | Total tweets: 7500
 >> Computing APriori on day: 2020-07-29 | Total tweets: 2780
 >> Computing APriori on day: 2020-07-30 | Total tweets: 1980
 >> Computing APriori on day: 2020-07-31 | Total tweets: 7500
 >> Computing APriori on day: 2020-08-01 | Total tweets: 7500
 >> Computing APriori on day: 2020-08-02 | Total tweets: 7500
 >> Computing APriori on day: 2020-08-03 | Total tweets: 0
 >> Computing APriori on day: 2020-08-04 | Total tweets: 7500
 >> Computing APriori on day: 2020-08-05 | Total tweets: 0
 >> Computing APriori on day: 2020-08-06 | Total tweets: 7214
 >> Computing APriori on day: 2020-08-07 | Total tweets: 1060
 >> Computing APriori on day: 2020-08-08 | Total tweets: 7500
 >> Computing 

In [13]:
print(" > Elapsed time:", "%.2f" % round(eff_apriori_time, 2), "seconds")

 > Elapsed time: 65.03 seconds


### 2. Check if the groups appear in different dates

In [14]:
final_results = {}

# iterate on every [date, [[group_of_2],[group_of_3]]]
for res in results_list:

    date = res[0]           # get the date
    list_of_groups = res[1] # in pos 0 --> 1 word, in pos 1 --> 2 words, ...

    # iterate on every pair of every groups

    for group in list_of_groups:
        for key, value in group.items():

            # if the key is already in the solutions dict --> update its value with current data 
            if key in final_results:
                final_results[key][0].append(date)
                final_results[key][1].append(value)

            # if it is a new group --> create a list with the date value
            else:
                final_results[key] = [[date],[value]]

print(" > Operation Completed")

 > Operation Completed


### Store the data in a dataframe

In [15]:
words_column = []
dates_column = []
count_column = []
for k,v in final_results.items():
    words_column.append(k)
    dates_column.append(v[0])
    count_column.append(v[1])

final_results_df = pd.DataFrame(
    {'group_of_words': words_column,
     'dates': dates_column,
     'supports': count_column
    })

final_results_df

,group_of_words,dates,supports
0,"(good,)","[2020-07-24, 2020-07-25, 2020-07-28, 2020-08-0...","[0.020338983050847456, 0.015283454771636751, 0..."
1,"(live,)","[2020-07-24, 2020-07-25, 2020-07-26, 2020-07-2...","[0.02711864406779661, 0.019489366743676323, 0...."
2,"(covid,)","[2020-07-24, 2020-07-25, 2020-07-26, 2020-07-2...","[0.5728813559322034, 0.6070730406966411, 0.618..."
3,"(coronaviru,)","[2020-07-24, 2020-07-25, 2020-07-26, 2020-07-2...","[0.0847457627118644, 0.08601386173804869, 0.1,..."
4,"(pandem,)","[2020-07-24, 2020-07-25, 2020-07-26, 2020-07-2...","[0.06440677966101695, 0.0444286475919673, 0.04..."
...,...,...,...
1470,"(hour, last)",[2020-08-30],[0.01528358208955224]
1471,"(covid, india, new)",[2020-08-30],[0.016955223880597014]
1472,"(case, india, new)",[2020-08-30],[0.017194029850746268]
1473,"(case, covid, death, new)",[2020-08-30],[0.020417910447761194]


### 3. Remove the rows with less than 2 dates
We are not interested in them, they are just frequent topics, not topics frequent in time.

In [16]:
count_dates = []
for l in final_results_df["dates"]:
    count_dates.append(len(l))

final_results_df["tot_dates"] = count_dates
final_results_df = final_results_df.loc[final_results_df['tot_dates'] > 1]
final_results_df = final_results_df.sort_values(by=['tot_dates'], ascending=False)

final_results_df

,group_of_words,dates,supports,tot_dates
54,"(posit,)","[2020-07-24, 2020-07-25, 2020-07-26, 2020-07-2...","[0.020338983050847456, 0.04342159824654938, 0....",26
88,"(coronaviru, covid)","[2020-07-24, 2020-07-25, 2020-07-26, 2020-07-2...","[0.0576271186440678, 0.06125229548012558, 0.07...",26
33,"(go,)","[2020-07-24, 2020-07-25, 2020-07-26, 2020-07-2...","[0.04067796610169491, 0.02701261773591612, 0.0...",26
91,"(case, covid)","[2020-07-24, 2020-07-25, 2020-07-26, 2020-07-2...","[0.04067796610169491, 0.08749481665778094, 0.1...",26
31,"(get,)","[2020-07-24, 2020-07-25, 2020-07-26, 2020-07-2...","[0.04406779661016949, 0.03412120135063089, 0.0...",26
...,...,...,...,...
218,"(case, covid, day)","[2020-07-27, 2020-08-09]","[0.019333333333333334, 0.0212]",2
17,"(well,)","[2020-07-24, 2020-08-07]","[0.020338983050847456, 0.01509433962264151]",2
230,"(start,)","[2020-07-29, 2020-08-07]","[0.015827338129496403, 0.016037735849056604]",2
15,"(watch,)","[2020-07-24, 2020-07-28]","[0.01694915254237288, 0.015466666666666667]",2


## Save the results
both as .csv and .pkl

In [17]:
# save the data (could be useful in the future)
final_results_df.to_csv("../data/results/results_df.csv")
final_results_df.to_pickle("../data/results/results_df.pkl")

print(" > Results correctly stored in 'data/results'")

 > Results correctly stored in 'data/results'
